In [16]:
from music21 import *
%load_ext music21.ipython21

In [230]:
# simple notes show

def showNotes(note_sequece, time_meter, duration_divider):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='part0')
    p0.timeSignature = meter.TimeSignature(TS)
    for i, notes in enumerate(note_sequece):
        notes.quarterLength = notes.quarterLength*duration_divider
        p0.append(notes)
    s.insert(0, p0)
    return s

In [231]:
#### the primary axises can change but this class describes 1 PA for now
#### so 0 can vary!
# what kind of data structure should i use?? scale?

# primary axis PA
# motions in PA = 0,A,B,C and combinations of them
# AXES:
# 0 = no motion = absolute balance
# A = Accending from PA = unbalancing (leading away from 0)
# B = Descending toward PA = balancing (leading to 0)
# C = Accending towards PA = balancing (leading to 0)
# D = Descending from PA = unbalancing (leading away from 0)
# VARIABLES:
# two variables = P for pitch and T for time
# f.e.: ATP
# f.e.2: A4T3P

# generate PA class
# 4 args scale, motion sequence, P, T
# take PAaxes combination f.e. [2xA 1xB 3xD 1xC] or [AABDDDC]
# take variables f.e. P = [2,1,1,2]
# P: f.e. P = [2,1,1,2]
# T: f.e. T = [3,1,2,2,2,3] or so
# combine PAaxes with P and T
# return PA melody

class PrimaryAxis:
    
    sequence_list = [] # not needed in return
    durations = [] # return
    pitches = [] # return
    seq_length = 0 # return
    
    def __init__(self, scale, motion_sequence, T, P):
        
        self.scale = scale
        self.motion_sequence = motion_sequence
        self.P = P
        self.T = T
        
        self.combinePA()
        self.calculate_sequence()
        self.seq_length = len(self.durations)
    
    def combinePA(self):
        
        for i, e in enumerate(self.motion_sequence):
            sequence = self.motion_sequence[i],self.T[i%len(self.T)],self.P[i%len(self.P)]
            self.sequence_list.append(sequence)
        return
    
    def calculate_sequence(self):
        for i, e in enumerate(self.sequence_list):
            if e[0] == "A":
                self.durations.append(e[1])
                self.pitches.append([0, 0 + e[2]])
            elif e[0] == "B":
                self.durations.append(e[1])
                self.pitches.append([0 + e[2], 0])
            elif e[0] == "C":
                self.durations.append(e[1])
                self.pitches.append([0 - e[2],0])
            elif e[0] == "D":
                self.durations.append(e[1])
                self.pitches.append([0, 0 - e[2]])

#motion_sequence = "AABDDDC"
motion_sequence = "D"* 2 + "A"*1 + "B"*1 + "D"*2 + "A"* 2 + "D"*1 + "B"*1 + "C"*2
time_seqence = [2,1,1]
pitch_sequence = [4,1,2,2,1,4]
PA = PrimaryAxis("amajor", motion_sequence, time_seqence, pitch_sequence)
PA.sequence_list, PA.durations, PA.pitches;

In [232]:
myscale = scale.HarmonicMinorScale('C4') # not needed for now

In [233]:
def convert_to_music21_sequence(sc, pitches, durations):
    note_seq = []
    tonic = sc.getTonic()
    for i, o in enumerate(pitches):
        for e, p in enumerate(o):
            c = sc.pitchFromDegree(p+1)
            
            nota = note.Note(c.nameWithOctave)
            if p >= 0:
                nota.octave = 4
            else:
                nota.octave = 3
            if e == 0:
                nota.duration.quarterLength = durations[i]
            else:
                nota.duration.quarterLength = 1
            note_seq.append(nota)
    return note_seq

In [234]:
pitches = convert_to_music21_sequence(myscale,PA.pitches, PA.durations)

In [236]:
s = showNotes(pitches,[4,4],2)
s.show('musicxml')

TypeError: can't multiply sequence by non-int of type 'list'

In [244]:
myscale.show('txt')

<music21.scale.HarmonicMinorScale C harmonic minor>


In [247]:
[myscale.pitchFromDegree(x+1).name for x in range(7)]

['C', 'D', 'E-', 'F', 'G', 'A-', 'B']

In [ ]:
#sp = midi.realtime.StreamPlayer(s)
#sp.play()
#s.show('midi')
#s.show('musicxml')
#s.show()
#scales